In [30]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import datetime
import random
import re
import pprint
import json
import pandas as pd

def getMatchLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk/rugby/fixtures/_/date/" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)

    json_out = json.loads(json_text)
    
    # Get just the match links
    json_match = json_out['schedule']['groups']
    matches = []
    for m in json_match:
        for c in m['complete']:
            matches.append(c['result']['href'])
            
    return matches

#links = getMatchLinks('20170304')

In [32]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        json_out.append(json.loads(json_text))

    for j in json_out[0]['gamePackage']['links']:
        if j['pageType'] == 'matchstats':
            match_s = j['href']
        elif j['pageType'] == 'playerstats':
            player_s = j['href']
        else:
            match_s, player_s = "", ""
            
    return match_s, player_s

m, p = getStatsLinks(links[0])
print(m)
print(p)

In [33]:
def getStats(statsUrl):
    html = urlopen("http://www.espn.co.uk" + statsUrl)
    bsObj = bs(html, "lxml")  
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    stat_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        stat_out.append(json.loads(json_text))
        
    return stat_out[0]

In [34]:
def getPlayerVals(d):

    dct = d
    dct['match_date'] = date
    dct['match_id'] = match_id[0] # regex above returns list of one element
    if 'id' in dct:
        dct['player_id'] = dct.pop('id') # rename 'id' to 'player_id'
    # This value should be unique and will be the primary key of the table
    dct['row_nk'] = dct['player_id'] + dct['match_id'] 

    # remove entries we don't want - url is useless and eventTimes doesn't conform to the required structure
    # these are the keys, i.e. table column names
    cols = list(dct.keys())
    cols = list(filter(lambda c: c != 'eventTimes', cols))
    cols = list(filter(lambda c: c != 'url', cols))

    # these are used for the SQL query, so everything gets inserted at once
    placeholders = ', '.join(['%s'] * len(cols))
    columns = ', '.join(cols)

    # These are the values to be inserted into the table. Sometimes the values is a single value
    # and other times is a dictionary, handle both instances
    vals = []
    for d in dct:
        if d in cols:
            if type(dct[d]) is dict:
                vals.append(dct[d]['value'])
            else:
                vals.append(dct[d])

    return placeholders, columns, vals

In [39]:
# query to create the table in MySQL - this will need a match and player identifier
sql = """drop table if exists scraping.p_stats;
        create table p_stats( row_nk varchar(50) not null
                            , player_id bigint(7) not null
                            , match_id varchar(20) not null
                            , match_date varchar(20)
                            , name varchar(200)
                            , number varchar(10)
                            , position varchar(10)
                            , captain varchar(10)
                            , subbed varchar(10)
                            , homeAway varchar(10)
                            , subToolTip varchar(10)
                            , onPitch varchar(10)
                            , wasActive varchar(10)
                            , tries bigint(7)
                            , tryassists bigint(7)
                            , points bigint(7)
                            , kicks bigint(7)
                            , passes bigint(7)
                            , runs bigint(7)
                            , metres bigint(7)
                            , cleanbreaks bigint(7)
                            , defendersbeaten bigint(7)
                            , offload bigint(7)
                            , lineoutwonsteal bigint(7)
                            , turnoversconceded bigint(7)
                            , tackles bigint(7)
                            , missedtackles bigint(7)
                            , lineoutswon bigint(7)
                            , penaltiesconceded bigint(7)
                            , yellowcards bigint(7)
                            , redcards bigint(7)
                            , penalties bigint(7)
                            , penaltygoals bigint(7)
                            , conversiongoals bigint(7)
                            , dropgoalsconverted bigint(7)
                            , primary key(row_nk));""".replace("\n", "")

# Run this first to set up connection drop the table, and create again before putting more data into it

# Set up the connection
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port = 3306,
                       user = 'root', passwd = '', db = 'mysql')

cur = conn.cursor()
cur.execute("USE scraping")
cur.execute(sql)

0

In [40]:
dt = pd.date_range('20160801','20170317')
#dt = pd.date_range('20170304','20170305')
dates = dt.format(formatter=lambda x: x.strftime('%Y%m%d'))

In [ ]:
teams, players = ['home', 'away'], ['team', 'reserves']
# dates = ['20170304', '20170305'] # Testing with two dates

for date in dates:
    matches = getMatchLinks(date)
    
    print(date)
    print(matches, len(matches))
    
    if (len(matches) > 0): # Some days have no matches => no links
        for match_link in matches:
            if match_link != '#': # Some games have no links so populate list with '#'
                ma, pl = getStatsLinks(match_link)

                regex = re.compile('gameId=([0-9]*)')
                match_id = regex.findall(match_link)

                if pl != "":
                    p_stats = getStats(pl)

                    for t in teams:
                        for p in players:
                            data = p_stats["gamePackage"]["matchLineUp"][t][p]
                            for d in data:               
                                placeholders, columns, vals = getPlayerVals(d)

                                cur = conn.cursor()
                                cur.execute("USE scraping")

                                # Execute the SQL to update the table
                                sql_update = "insert into p_stats (%s) values (%s)" % (columns, placeholders)
                                cur.execute(sql_update, vals)
                                conn.commit()

cur.close()
conn.close()

20160801
[] 0
20160802
[] 0
20160803
[] 0
20160804
[] 0
20160805
['/rugby/match?gameId=290127&league=270555', '/rugby/match?gameId=290126&league=270555'] 2
20160806
['/rugby/match?gameId=289960&league=283', '/rugby/match?gameId=289959&league=283', '/rugby/match?gameId=289958&league=283', '/rugby/match?gameId=289957&league=283', '/rugby/match?gameId=289956&league=283', '/rugby/match?gameId=289955&league=283', '/rugby/match?gameId=289954&league=283', '/rugby/match?gameId=289953&league=283', '/rugby/match?gameId=289952&league=283', '/rugby/match?gameId=289951&league=283', '/rugby/match?gameId=289950&league=283', '/rugby/match?gameId=289949&league=283', '/rugby/report?gameId=276667&league=242041', '/rugby/match?gameId=290125&league=270555', '/rugby/match?gameId=289863&league=264129'] 15
20160807
['/rugby/match?gameId=289972&league=283', '/rugby/match?gameId=289971&league=283', '/rugby/match?gameId=289970&league=283', '/rugby/match?gameId=289969&league=283', '/rugby/match?gameId=289968&leag

In [38]:
# Run this if the code fails to close the DB connection
cur.close()
conn.close()